In [1]:
# Bibliotecas necessárias para o experimento

import numpy as np
from random import randint

In [2]:
def S(Ut, Uf, X):
    W = np.dot(np.transpose(Uf), Ut)
    T = np.transpose(W)
    A = np.dot(np.transpose(Ut), np.transpose(X))
    C = np.dot(A, np.transpose(Uf))
    L = W - np.identity(3) - 1/2*(W - np.identity(3))**2
    S = 2*np.trace(np.dot((-np.dot(T, C) + 2*C),L))
    #S = 2*np.trace(np.dot((-1/2*np.dot(C, T) - 1/2*np.dot(T, C) + 2*C),L))
    return S

In [3]:
def field(Ut, Uf, a, h, A, B):
    SA = S(Ut, Uf, A)
    SB = S(Ut, Uf, B)
    f = -a*SA-h*(SB/SA)
    return f

In [4]:
def lind(u, Ut, Uf, t, lbda):
    Ax = [[0, 0, 0],[0, 0, -1], [0, 1, 0]]
    Ay = [[0, 0, 1],[0, 0, 0], [-1, 0, 0]]
    Az = [[0, -1, 0],[1, 0, 0], [0, 0, 0]]
    B = [[-1*lbda, 0, 0],[0, -1*lbda, 0], [0, 0, -2*lbda]]
    if t == 0:
        fx=10.28
        fy=10.73
        fz=40
        F=np.dot(fx,Ax)+np.dot(fy,Ay)+np.dot(fz,Az)+B
    else:
        fx=field(Ut, Uf, u[0], u[3], Ax, B)
        fy=field(Ut, Uf, u[1], u[4], Ay, B)
        fz=field(Ut, Uf, u[2], u[5], Az, B)
        P=400

        if abs(fx) > P:
            if fx < 0:
                fx = -P
            else:
                fx = P
        if abs(fy) > P:
            if fy < 0:
                fy = -P
            else:
                fy = P
        if abs(fy) > P:
            if fy < 0:
                fy = -P
            else:
                fy = P

        F=np.dot(fx,Ax)+np.dot(fy,Ay)+np.dot(fz,Az)+B
        	
    return fx,fy,fz,F

In [5]:
def rungeKutta(u, Ut, Uf, h, t, lbda):
    fx,fy,fz,k1 = lind(u, Ut, Uf, t, lbda)
    k1 = np.dot(k1,Ut)
    fx,fy,fz,k2 = lind(u, (Ut+h/2*k1), Uf, t, lbda)
    k2 = np.dot(k2,(Ut+h/2*k1))
    fx,fy,fz,k3 = lind(u, (Ut+h/2*k2), Uf, t, lbda)
    k3 = np.dot(k3,(Ut+h/2*k2))
    fx,fy,fz,k4 = lind(u, (Ut+h*k3), Uf, t, lbda)
    k4 = np.dot(k4,(Ut+h*k3))
    a = fx;
    b = fy;
    c = fz;
    result = Ut+(h/6)*(k1+2*k2+2*k3+k4);
    return a,b,c,result

In [6]:
u = [70, 106, 66, 0.35, 0.31, 0.34] # Hadamard
#u = [26, 141, 74, 0.35, 0.31, 0.34] # Pauli-X
#u = [102, 13, 84, 0.35, 0.31, 0.34] # Pauli-Y
#u = [199, 125, 18, 0.35, 0.31, 0.34] # Pauli-Z
#u = [randint(1,400), randint(1,400), randint(1,400), 0.35, 0.31, 0.34] # Phase(S,P)
#u = [330, 66, 43, 0.35, 0.31, 0.34]  # Porta T
print(f'u = {u}')

Ut = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
Uf = [[0, 0, 1], [0, -1, 0], [1, 0, 0]] # Hadamard
#Uf = [[1, 0, 0], [0, -1, 0], [0, 0, -1]] # Pauli-X
#Uf = [[-1, 0, 0], [0, 1, 0], [0, 0, -1]] # Pauli-Y
#Uf = [[-1, 0, 0], [0, -1, 0], [0, 0, 1]] # Pauli-Z
#Uf = [[0, -1, 0],[-1, 0, 0], [0, 0, 1]] # Phase(S,P)
#Uf = [[1/np.sqrt(2), -1/np.sqrt(2), 0], [-1/np.sqrt(2), 1/np.sqrt(2), 0], [0, 0, 1]] # Porta T

h=0.001
t=0
i=0
lbda=0.01
distance_vector = []
   
while t <= 100:
    a,b,c,Ut = rungeKutta(u, Ut, Uf, h, t, lbda)
    D = np.linalg.norm(Ut-Uf)**2
    distance_vector.append(D)
    t += 1

r0 = [[1], [0], [0]]
rf = np.dot(Uf, r0)
print(f'O rf desejado é: {rf}')
pf = [[1/2*(1+rf[2][0]), 1/2*(rf[0][0]-1j*rf[1][0])], [1/2*(rf[0][0]+1j*rf[1][0]), 1/2*(1-rf[2][0])]]
print(f'O rho desejado é: {pf}')

rt = np.dot(Ut, r0)
print(f'O vetor de Bloch encontrado é: {rt}')
pt = [[1/2*(1+rt[2][0]), 1/2*(rt[0][0]-1j*rt[1][0])], [1/2*(rt[0][0]+1j*rt[1][0]), 1/2*(1-rt[2][0])]]
print(f'O rho alcançado foi: {pt}')
print(f'A fidelidade do experimento é de: {np.trace(np.sqrt(np.sqrt(pt)*pf*np.sqrt(pt)))**2}')
print(f'O vetor U alcançado foi: {Ut}')
print(f'A distância encontrada, ao final do experimento foi: {D}')

u = [70, 106, 66, 0.35, 0.31, 0.34]
O rf desejado é: [[0]
 [0]
 [1]]
O rho desejado é: [[1.0, 0j], [0j, 0.0]]
O vetor de Bloch encontrado é: [[ 0.00615211]
 [-0.00253629]
 [ 0.99255554]]
O rho alcançado foi: [[0.9962777706911535, (0.003076055334436111+0.0012681439675681185j)], [(0.003076055334436111-0.0012681439675681185j), 0.0037222293088464697]]
A fidelidade do experimento é de: (0.9962777706911535+0j)
O vetor U alcançado foi: [[ 6.15211067e-03 -1.04933434e-03  9.92588646e-01]
 [-2.53628794e-03 -9.92223587e-01  2.94385972e-04]
 [ 9.92555541e-01  3.72011791e-03  4.66900645e-03]]
A distância encontrada, ao final do experimento foi: 0.00025192861114537005


In [7]:
u = [70, 106, 66, 0.35, 0.31, 0.34] # Hadamard
#u = [26, 141, 74, 0.35, 0.31, 0.34] # Pauli-X
#u = [102, 13, 84, 0.35, 0.31, 0.34] # Pauli-Y
#u = [199, 125, 18, 0.35, 0.31, 0.34] # Pauli-Z
#u = [randint(1,400), randint(1,400), randint(1,400), 0.35, 0.31, 0.34] # Phase(S,P)
#u = [330, 66, 43, 0.35, 0.31, 0.34]  # Porta T
print(f'u = {u}')

Ut = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
Uf = [[0, 0, 1], [0, -1, 0], [1, 0, 0]] # Hadamard
#Uf = [[1, 0, 0], [0, -1, 0], [0, 0, -1]] # Pauli-X
#Uf = [[-1, 0, 0], [0, 1, 0], [0, 0, -1]] # Pauli-Y
#Uf = [[-1, 0, 0], [0, -1, 0], [0, 0, 1]] # Pauli-Z
#Uf = [[0, -1, 0],[-1, 0, 0], [0, 0, 1]] # Phase(S,P)
#Uf = [[1/np.sqrt(2), -1/np.sqrt(2), 0], [-1/np.sqrt(2), 1/np.sqrt(2), 0], [0, 0, 1]] # Porta T

h=0.001
t=0
i=0
lbda=0.01
distance_vector = []
   
while t <= 100:
    a,b,c,Ut = rungeKutta(u, Ut, Uf, h, t, lbda)
    D = np.linalg.norm(Ut-Uf)**2
    distance_vector.append(D)
    t += 1

r0 = [[1], [0], [0]]
rf = np.dot(Uf, r0)
print(f'O rf desejado é: {rf}')
pf = [[1/2*(1+rf[2][0]), 1/2*(rf[0][0]-1j*rf[1][0])], [1/2*(rf[0][0]+1j*rf[1][0]), 1/2*(1-rf[2][0])]]
print(f'O rho desejado é: {pf}')

rt = np.dot(Ut, r0)
print(f'O vetor de Bloch encontrado é: {rt}')
pt = [[1/2*(1+rt[2][0]), 1/2*(rt[0][0]-1j*rt[1][0])], [1/2*(rt[0][0]+1j*rt[1][0]), 1/2*(1-rt[2][0])]]
print(f'O rho alcançado foi: {pt}')
print(f'A fidelidade do experimento é de: {np.trace(np.sqrt(np.sqrt(pt)*pf*np.sqrt(pt)))**2}')
print(f'O vetor U alcançado foi: {Ut}')
print(f'A distância encontrada, ao final do experimento foi: {D}')

u = [70, 106, 66, 0.35, 0.31, 0.34]
O rf desejado é: [[0]
 [0]
 [1]]
O rho desejado é: [[1.0, 0j], [0j, 0.0]]
O vetor de Bloch encontrado é: [[ 0.00615211]
 [-0.00253629]
 [ 0.99255554]]
O rho alcançado foi: [[0.9962777706911535, (0.003076055334436111+0.0012681439675681185j)], [(0.003076055334436111-0.0012681439675681185j), 0.0037222293088464697]]
A fidelidade do experimento é de: (0.9962777706911535+0j)
O vetor U alcançado foi: [[ 6.15211067e-03 -1.04933434e-03  9.92588646e-01]
 [-2.53628794e-03 -9.92223587e-01  2.94385972e-04]
 [ 9.92555541e-01  3.72011791e-03  4.66900645e-03]]
A distância encontrada, ao final do experimento foi: 0.00025192861114537005
